# Поиск фактов о персонах
## Как работает
1. На вход подаются массив неразмеченных текстов новостей. Как формирует массив читать ___
2. При помощи библиотеки **natasha** массив последовательно предобрабатывается. В результате получаем последовательность рахмеченных документов
3. Каждый документ разбивается на предложения, поиск фактов выполняется по каждому предложению
## Правила поиска фактов  
Для сущности персона описано несколько правил, определяющих различные типы фактов на основании синтаксического разбора предложения 
- **nsubj** - дествие (глагол) которое совешила персона "Ширак *РОДИЛСЯ* в Париже" (кто, что сделал, где)
- **appos** - характеристика, кем является данная персона "при *ПРЕЗЕДЕНТЕ* России Ельцине" (кто, кем является, подробнее)
- **obj** - действие, которое совершено над персоной "суд *ПРИЗНАЛ* Навального виновным" (кто, что с ним сделали, подробнее)
- **iobj** - дествие (причастие) которое совешила персона "Порошенко *ПРОИГРАВ* выборы" (кто, что им сделано, подробнее)
- **conj** - вторая персона "Навальный *и Офицеров* (кто и кто)"
- **nmod** - что-то относится к персоне "*ФОТОГРАФИЯ* Дональда Трампа" (кто, что относится к нему)


In [ ]:
# Разбор (разметка) корпуса текстов через natasha
# на входе - корпус исходных текстов (калатог в котором 1 файл - 1 статья источника-сайта)
# на выходе - ______________
from pathlib import Path
from ipymarkup import show_box_markup
#from anytree import Node, RenderTree, resolver, walker
#import networkx
#import matplotlib.pyplot as plt
from treelib import Node, Tree, exceptions

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    #PER,
    NamesExtractor,
    #AddrExtractor,
    #DatesExtractor,
    #MoneyExtractor,

    Doc
)

def main (corpus_path: str):
    # !!! внимание. исходные файлы должны быть сохранение в определенном формате
    paths = Path(corpus_path).glob('*.txt')
    COUNTER = 30
    for filename in paths:
        if not COUNTER:
            break
        with open(filename, encoding='utf-8') as fdr:
            create_date, href, title, text = [line.strip('\n') for line in fdr][:4]

            segmenter = Segmenter()
            morph_vocab = MorphVocab()

            emb = NewsEmbedding()
            morph_tagger = NewsMorphTagger(emb)
            syntax_parser = NewsSyntaxParser(emb)
            ner_tagger = NewsNERTagger(emb)
            
            doc = Doc(text)
            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            doc.parse_syntax(syntax_parser)
            doc.tag_ner(ner_tagger)
            

            for token in doc.tokens:
                token.lemmatize(morph_vocab)

            for span in doc.spans:
                span.normalize(morph_vocab)

            for sent in doc.sents[:1]:
                for span in sent.spans:
                    for _ in sent.tokens:
                        if span.tokens[0].id == _.id:
                            s= {'type': span.type, 'text':_.text, 'rel':_.rel, 'head':_.head_id }  
                    for _ in sent.tokens:
                        if _.id == s['head']:
                            s['head'] = _.text
                    if s['type'] == 'PER':      
                        print (sent.text)
                        print(s)
                        print('------------------')
            
            '''
            for sent in doc.sents:
                # по результату синтаксического разбора создаем дерево
                tree = Tree()
                tree.create_node('root', 'root')
                
                nodes = [tree.create_node(_.text, _.id, parent='root', 
                        data={'head_id':_.head_id, 'start':_.start, 'rel':_.rel, 'pos':_.pos}) 
                        for _ in sent.tokens]

                for node in nodes:
                    if node.identifier == node.data['head_id'] or not tree.get_node(node.data['head_id']):
                        continue
                    if node.data['rel'] == 'root':
                        node.root = True
                    try:
                        tree.move_node(node.identifier, node.data['head_id'])
                    except exceptions.LoopError:
                        print ('loop')

                print('-------------------')
                print (sent.text)
            '''
            '''
                https://universaldependencies.org/u/dep/index.html
                источник -отношение-> цель
                отнонение указывается в источнике но относится к цели
                
                obj - цель = объект, на который воздействует источник 
                nsubj - цель = источник относится ко мне
                obl - цель = когда
                nmod - цель = я принадлежу источнику
                appos - цель = что из себя представляет источник

                правила 
                PER
                - nsubj - дествие (глагол) которое совешила персона "Ширак РОДИЛСЯ в Париже" (кто, что сделал, где)
                - appos - характеристика, кем является данная персона "при ПРЕЗЕДЕНТЕ России Ельцине" (кто, кем является, подробнее)
                - obj - действие, которое совершено над персоной "суд ПРИЗНАЛ Навального виновным" (кто, что с ним сделали, подробнее)
                - iobj - дествие (причастие) которое совешила персона "Порошенко ПРОИГРАВ выборы" (кто, что им сделано, подробнее)
                - conj - вторая персона "Навальный и Офицеров (кто и кто)"
                - nmod - что-то относится к персоне "ФОТОГРАФИЯ Дональда Трампа" (кто, что относится к нему)

            '''
            '''

                spans_id = {_.tokens[0].id: {'type':_.type, 'text':_.text, 'type':_.type} for _ in sent.spans}
                for path in tree.paths_to_leaves():
                    if set(path) & set(spans_id.keys()):
                        result = [[tree[_].tag, tree[_].data["pos"], tree[_].data["rel"]]  for _ in path[1:]]
                        for span_key, span_data in spans_id.items():
                            if span_key in path:
                                result = [span_data['text'], span_data['type']] + result
                        print (result)


                        #s = ' '.join([f'{tree[_].tag}/({tree[_].data["pos"]})/{tree[_].data["rel"]} ' for _ in path[1:]])
                        #print (s)
            '''         
            '''
            # !!! дерево построит нельзя потому что синтаксическое предложение в общем случае имеет циклы - это граф
            finder = resolver.Resolver('name')
            finder_start = resolver.Resolver('start')

            for sent in doc.sents[:1]:
              root = Node('#')
              nodes = [Node(_.id, parent = root, parent_id=_.head_id, text=_.text, rel=_.rel, start=str(_.start)) for _ in sent.tokens]
              for node in nodes:
                try:
                    node.parent = finder.get(root, node.parent_id)
                except:
                    pass
              print ('---------------------')
              print (RenderTree(root))
              for span in [_.start for _ in sent.spans]:
                print (finder_start.get(root, f'{span}'))
              root = None
            '''
            '''
            # объект - описание - дополнение
            # Сергей Пугачев Бенефициар Межпромбанка 
            for sent in doc.sents[:1]:
                words = [_.text for _ in sent.tokens]
                spans = [(_.start, _.stop, _.type) for _ in sent.spans]
                deps = [(int(_.head_id.split('_')[-1])-1, int(_.id.split('_')[-1])-1, _.rel) for _ in sent.tokens]
                
                spans2 = {f'{_.start}': _.text for _ in sent.spans}
                words2 = {f'{_.id}': (_.text,  _.start) for _ in sent.tokens}
                deps2 = {f'{_.start}': f'{_.head_id}' for _ in sent.tokens}
                deps2_reverse = {v:k for k,v in deps2.items()}
                for sp in spans2:
                    s= f'{spans2[sp]}'
                    deps2[sp]
                    t, st = words2[deps2[sp]]
                    s += f' {t}'
                    
                    print (s)                 
                
                #print (spans2)
                #print (words2)
                #print (deps2)
                #print ('---------------')

                #show_box_markup(sent.text, sent.spans)
            '''
            COUNTER -= 1


CORPUS_PATH = 'C:\\tmp\\docs\\#learning\\article\\Python\\example\\corpus\\rbc'
main(CORPUS_PATH)